In [50]:
import pricing
import model_calibration as model
import options as opt
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
from datetime import datetime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# S0 = 100
ticker='AAPL'
S0=yf.download(ticker).iloc[-1]['Adj Close']
K = 250
T = 1
M = 100000
opt_type = 'C'

# Get risk-free rate
rates=model.get_riskfree_rate()
r=rates[str(T*12)]/100

# Get the implied volatility
volatility_surface=model.volatility_surface(ticker)

maturities=volatility_surface['Time to maturity'].values
strikes=volatility_surface['Strikes'].values
volatilities=volatility_surface['Implied volatility'].values

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [45]:
points=np.array([maturities, strikes]).T
IV=griddata(points, volatilities, (T, K), method='linear').item()/100


In [46]:
price=pricing.vanilla_option_price(S0, K, T, r, IV, M, opt_type, antithetic=True)

In [47]:
price

20.14426668326883

In [4]:
v=model.volatility_surface('AAPL')

[*********************100%%**********************]  1 of 1 completed


In [7]:
v

,Time to maturity,Strikes,Implied volatility
0,0.010959,100.0,30.000000
1,0.010959,115.0,30.000000
2,0.010959,130.0,30.000000
3,0.010959,135.0,30.000000
4,0.010959,140.0,30.000000
...,...,...,...
966,2.139726,410.0,22.814246
967,2.139726,420.0,23.255147
968,2.139726,430.0,23.447591
969,2.139726,440.0,23.560325


In [48]:
price=model.get_price('AAPL', 250, 1, pricing.vanilla_option_price)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [49]:
price

19.360176289376216

In [ ]:
def volatility_surface(ticker, r=0.02, price=False, method="optimizer"):
    """
    method : "optimizer" or "newton"
    """
    df=yf.download(ticker, start='2024-06-01')
    df.dropna(inplace=True)
    df.ffill(inplace=True)
    #today=datetime.now().date()
    #S=df.loc[today, 'Close']
    S=df['Close'].iloc[-1]
    exp_dates=opt.get_options_dates(ticker)
    time_to_maturity=[]
    strike=[]
    implied_volatility=[]
    for date in exp_dates:
        strikes=opt.call(ticker, date)['strike']
        option_prices=opt.call(ticker, date)['lastPrice']
        days_to_maturity=(datetime.strptime(date, '%Y-%m-%d').date()-datetime.now().date()).days
        maturity=days_to_maturity/365
        for i in range (len(strikes)):
            time_to_maturity.append(maturity)
            strike.append(strikes.iloc[i])
            if maturity>0:
                if method=="optimizer":
                    implied_volatility.append(model.implied_vol(S, strikes.iloc[i], maturity, r, option_prices.iloc[i]))
                elif method=="newton":
                    implied_volatility.append(model.implied_vol_newton(S, strikes.iloc[i], maturity, r, option_prices.iloc[i]))
                else:
                    print("method should be either 'optimizer' or 'newton'")
            else:
                implied_volatility.append(np.nan)    ##l'option a expiré donc il n'y a pas de volatilité implicite
        
    data=pd.DataFrame({'Time to maturity':time_to_maturity, 'Strikes':strike, 'Implied volatility':implied_volatility})
    if price:
        return data, S
    else:
        return data

In [ ]:
ticker_symbol='AAPL'

S=yf.download(ticker_symbol, start='2024-06-01')['Close'].iloc[-1]
#today=datetime.now().date()
#S=df.loc[today, 'Close']
ticker=yf.Ticker(ticker_symbol)
exp_dates=ticker.options
for date in exp_dates:
    call_options=ticker.option_chain(date).calls[['strike', 'lastPrice']]
    days_to_maturity=(datetime.strptime(date, '%Y-%m-%d').date()-datetime.now().date()).days
    # Conversion in years
    maturity=days_to_maturity/365
    print(len(call_options['strike']))

[*********************100%%**********************]  1 of 1 completed


52
47
31
75
30
12
76
57
57
47
54
28
47
59
54
62
62
61
60


In [87]:
def volatility_surface_v2(ticker, r=0.02, method="optimizer"):
    ticker=yf.Ticker(ticker_symbol)
    S=ticker.history(period="1d")['Close'].iloc[-1]
    exp_dates=ticker.options
    df=pd.DataFrame({'exp_dates':exp_dates})
    df['exp_dates'] = pd.to_datetime(df['exp_dates'], format='%Y-%m-%d')
    df['maturity'] = (df['exp_dates'] - pd.Timestamp.now()).dt.days / 365
    df['exp_dates']=df['exp_dates'].dt.strftime('%Y-%m-%d')

    def get_calls_by_date(ticker, date):
        return ticker.option_chain(date).calls[['strike', 'lastPrice']]

    option_data = []
    for _, row in df.iterrows():
        options = get_calls_by_date(ticker, row['exp_dates'])
        options['maturity'] = row['maturity']
        option_data.append(options)

    #option_data = [get_calls_by_date(ticker, date) for date in df['exp_dates']]

    result = pd.concat(option_data, ignore_index=True)
    #result['maturity'] = np.concatenate([np.repeat(maturity, len(options)) for maturity, options in zip(df['maturity'], option_data)])

    result['implied vol'] = result.apply(lambda row: model.implied_vol(S, row['strike'], row['maturity'], r, row['lastPrice']), axis=1)

    return result[['maturity', 'strike', 'implied vol']]

In [90]:
volatility_surface_v2('AAPL')

,maturity,strike,implied vol
0,0.002740,100.0,30.000000
1,0.002740,115.0,30.000000
2,0.002740,130.0,30.000000
3,0.002740,135.0,30.000000
4,0.002740,140.0,30.000000
...,...,...,...
966,2.131507,410.0,22.537965
967,2.131507,420.0,23.484048
968,2.131507,430.0,23.723413
969,2.131507,440.0,23.661834


In [89]:
model.volatility_surface('AAPL')

[*********************100%%**********************]  1 of 1 completed


,Time to maturity,Strikes,Implied volatility
0,0.005479,100.0,30.000000
1,0.005479,115.0,30.000000
2,0.005479,130.0,30.000000
3,0.005479,135.0,30.000000
4,0.005479,140.0,30.000000
...,...,...,...
966,2.134247,410.0,22.521737
967,2.134247,420.0,23.467222
968,2.134247,430.0,23.706471
969,2.134247,440.0,23.644979
